In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input/oct2017/OCT2017 /train"))

# Any results you write to the current directory are saved as output.

['DME', 'DRUSEN', 'CNV', 'NORMAL']


In [2]:
Training_Dir = '../input/oct2017/OCT2017 /train'
Validation_Dir = '../input/oct2017/OCT2017 /val'
Testing_Dir = '../input/oct2017/OCT2017 /test'

In [3]:
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

training_datagen = ImageDataGenerator(rescale = 1./255)
validation_datagen = ImageDataGenerator(rescale = 1./255)
testing_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = training_datagen.flow_from_directory(
	Training_Dir,
	target_size=(150,150),
  shuffle=True,
	class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
	Validation_Dir,
	target_size=(150,150),
  shuffle=True,
	class_mode='categorical'
)

testing_generator = testing_datagen.flow_from_directory(
	Testing_Dir,
	target_size=(150,150),
  shuffle=True,
	class_mode='categorical'
)

Found 83484 images belonging to 4 classes.
Found 32 images belonging to 4 classes.
Found 968 images belonging to 4 classes.


In [4]:
import numpy as np
np.unique(train_generator.classes)

array([0, 1, 2, 3], dtype=int32)

In [5]:
train_generator.class_indices, validation_generator.class_indices, testing_generator.class_indices

({'CNV': 0, 'DME': 1, 'DRUSEN': 2, 'NORMAL': 3},
 {'CNV': 0, 'DME': 1, 'DRUSEN': 2, 'NORMAL': 3},
 {'CNV': 0, 'DME': 1, 'DRUSEN': 2, 'NORMAL': 3})

In [6]:
np.unique(train_generator.classes, return_counts = True)

(array([0, 1, 2, 3], dtype=int32), array([37205, 11348,  8616, 26315]))

In [7]:
np.unique(testing_generator.classes, return_counts = True)

(array([0, 1, 2, 3], dtype=int32), array([242, 242, 242, 242]))

In [8]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')
])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 64)      1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 128)       147584    
__________

In [10]:
model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [11]:
history = model.fit_generator(train_generator, epochs=1, validation_data = testing_generator, verbose = 1)

Instructions for updating:
Use tf.cast instead.
2609/2609 [==============================] - 930s 356ms/step - loss: 0.5493 - acc: 0.8030 - val_loss: 0.1737 - val_acc: 0.9473


In [12]:
history = model.fit_generator(train_generator, epochs=2, validation_data = testing_generator, verbose = 1)

Epoch 1/2
2609/2609 [==============================] - 692s 265ms/step - loss: 0.3249 - acc: 0.8938 - val_loss: 0.1605 - val_acc: 0.9349
Epoch 2/2
2609/2609 [==============================] - 686s 263ms/step - loss: 0.3097 - acc: 0.9037 - val_loss: 0.1312 - val_acc: 0.9452


In [13]:
y_train_pred = model.predict_generator(train_generator)
y_train = train_generator.classes[train_generator.index_array]
y_train_pred = np.argmax(y_train_pred, axis=-1)

In [14]:
from sklearn.metrics import confusion_matrix
import itertools
cnf_matrix = confusion_matrix(y_train, y_train_pred)

In [15]:
cnf_matrix

array([[17519,  4423,  2788, 12475],
       [ 5381,  1364,   826,  3777],
       [ 4009,  1101,   627,  2879],
       [12412,  3090,  1983,  8830]])

In [16]:
from sklearn.metrics import accuracy_score
accuracy_score(y_train, y_train_pred)

0.3394662450289876

In [17]:
from sklearn.metrics import classification_report
target_names = ['CNV', 'DME', 'DRUSEN', 'NORMAL']

In [18]:
print(classification_report(y_train, y_train_pred, target_names=target_names))

              precision    recall  f1-score   support

         CNV       0.45      0.47      0.46     37205
         DME       0.14      0.12      0.13     11348
      DRUSEN       0.10      0.07      0.08      8616
      NORMAL       0.32      0.34      0.33     26315

   micro avg       0.34      0.34      0.34     83484
   macro avg       0.25      0.25      0.25     83484
weighted avg       0.33      0.34      0.33     83484

